1(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraElectra-cross-encoder-KD-v1)
2(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraElectra-cross-encoder-NoKD-v1)
3(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraDPR-cross-encoder-KD-v1)
4(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1)
5(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1)
6(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1)
7(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraElectra-bi-encoder-KD-v1)
8(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1)
9(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraDPR-bi-encoder-KD-v1)
10(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1)
11(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-KD-v1)
12(https://huggingface.co/hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1)

%env PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin
!pip install huggingface_hub -q
!pip install datasets -q
!pip install torch -q
!pip install transformers -q
!pip install tensorflow-cpu==2.16.1 -q
!pip install tf-keras==2.16.0 --no-dependencies -q
!pip install wandb -q
!pip install -U accelerate -q
!pip install -U transformers[torch] -q
!pip install sentence_transformers -q

In [3]:
!pip install sentence_transformers -q
!pip install huggingface_hub -q

In [19]:
organization_dataset_id="hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
qerl_file ="qrels.msmarco-passage.dev-subset.txt"
collection_name= "collection-arabic-preprocess-for-araelectra"
queries_name= "queries-arabic-preprocess-for-araelectra"
list_of_models= [ "hatemestinbejaia/mmarco-Arabic-AraElectra-bi-encoder-NoKD-v1", 
              "hatemestinbejaia/mmarco-Arabic-AraElectra-bi-encoder-KD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraDPR-bi-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraDPR-bi-encoder-KD-v1",
              "hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-KD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraElectra-cross-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraElectra-cross-encoder-KD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraDPR-cross-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-AraDPR-cross-encoder-KD-v1",
              "hatemestinbejaia/mmarco-Arabic-mMiniLML-cross-encoder-NoKD-v1",
              "hatemestinbejaia/mmarco-Arabic-mMiniLML-cross-encoder-KD-v1",
            ]
args_model= list_of_models[5]
print(args_model)
output_run= args_model.split("/")[1]
print(output_run)

hatemestinbejaia/mmarco-Arabic-mMiniLML-bi-encoder-KD-v1
mmarco-Arabic-mMiniLML-bi-encoder-KD-v1


In [5]:
#get the eval dataset
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=organization_dataset_id, filename=qerl_file, local_dir="./", repo_type="dataset")
# reranked 
ranked_list=["run.arabic-msmarco-passage.bm25tuned.txt"]
for i in range(len(ranked_list)):
    hf_hub_download(repo_id=organization_dataset_id, filename=ranked_list[i], local_dir="./", repo_type="dataset")

qrels.msmarco-passage.dev-subset.txt:   0%|          | 0.00/143k [00:00<?, ?B/s]

run.arabic-msmarco-passage.bm25tuned.txt:   0%|          | 0.00/131M [00:00<?, ?B/s]

In [21]:
import pandas as pd
#defaut number of condidat document K is 1000
def load_run(path, k=1000):
    print('Loading run...')
    run = pd.read_csv(path, delim_whitespace=True, header=None)
    run = run[run[2] <= k] 
    print(run.head(20))
    run = run.groupby(0)[1].apply(list).to_dict()
    return run
run = load_run(ranked_list[0], 1000)
output_run += ranked_list[0]+".tsv"
print(output_run)

Loading run...


/tmp/ipykernel_30/3188491716.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  run = pd.read_csv(path, delim_whitespace=True, header=None)


    0        1   2
0   2  1001873   1
1   2  4339073   2
2   2  4339075   3
3   2  4339072   4
4   2  6920058   5
5   2  7496504   6
6   2  6285817   7
7   2  4339068   8
8   2  7496506   9
9   2  2022779  10
10  2  2022784  11
11  2  4906202  12
12  2  1591543  13
13  2  3634076  14
14  2  7496508  15
15  2   385343  16
16  2  7496507  17
17  2  1782337  18
18  2  7496505  19
19  2  1571085  20
mmarco-Arabic-mMiniLML-bi-encoder-KD-v1run.arabic-msmarco-passage.bm25tuned.txt.tsv


In [7]:
from datasets import load_dataset, Dataset, load_from_disk
datasetQ = load_dataset(organization_dataset_id, queries_name, trust_remote_code=True)
print(datasetQ['dev'][1])
import tqdm
queries = {}
for i in tqdm.trange(len(datasetQ['dev'])):
    queries[datasetQ['dev'][i]["id"]]= datasetQ['dev'][i]["text"]
    #break

README.md:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/26.5M [00:00<?, ?B/s]

dev.full-00000-of-00001.parquet:   0%|          | 0.00/3.63M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/248k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/808731 [00:00<?, ? examples/s]

Generating dev.full split:   0%|          | 0/101093 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6980 [00:00<?, ? examples/s]

{'id': 1215, 'text': '3 مستويات للحكومة في كندا ومسؤولياتهم'}


100%|██████████| 6980/6980 [00:00<00:00, 10131.54it/s]


In [8]:
datasetC = load_dataset(organization_dataset_id, collection_name, trust_remote_code=True)
print(datasetC['collection'][100])
import tqdm
corpus = {}
for i in tqdm.trange(len(datasetC['collection'])):
    corpus[datasetC['collection'][i]["id"]]= datasetC['collection'][i]["text"]
    #break

collection-00000-of-00010.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

collection-00001-of-00010.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

collection-00002-of-00010.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

collection-00003-of-00010.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

collection-00004-of-00010.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

...

collection-00008-of-00010.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

collection-00009-of-00010.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

Generating collection split:   0%|          | 0/8841823 [00:00<?, ? examples/s]

{'id': 100, 'text': 'أنتونون دفوروك ( 1841 - 1904 ) كان أنتونين دفوراك ابن جزار ، لكنه لم يتبع تجارة والده . أثناء مساعدة والده بدوام جزئي ، درس الموسيقى وتخرج من مدرسة براغ أورغن في عام 1859 .'}


100%|██████████| 8841823/8841823 [13:55<00:00, 10586.89it/s]


In [9]:
import operator
import torch
def reranking_crossEncoder(query_id, passage_list_id):
    corpora=[]
    query = queries[query_id]
    X=[]
    for j in passage_list_id : 
        corpora.append([query, corpus[j]])
        X.append(j)
    with torch.no_grad():scores = model.predict(corpora, batch_size=8, show_progress_bar=False)
    #print(scores)
    x = dict(zip(X, scores))
    #print(x)
    xr = dict(sorted(x.items(), key=operator.itemgetter(1), reverse=True))
    #print("***************************************")
    #print("***************************************")
    #print(xr)
    #print(xr.keys())
    return list(xr.keys())

In [10]:
def reranking_biEncoder(query_id, passage_list_id):
    corpora=[]
    query = queries[query_id]
    X=[]
    for j in passage_list_id : 
        corpora.append(corpus[j])
        X.append(j)
    with torch.no_grad():query_embedding = model.encode(query, convert_to_tensor=True, show_progress_bar=False)
    with torch.no_grad(): corpus_embeddings = model.encode(corpora, convert_to_tensor=True, show_progress_bar=False, batch_size=128)
    scores=[]
    for i in corpus_embeddings:
        #print(corpus[i])
        cos_scores = util.cos_sim(query_embedding, i)[0]
        #print(cos_scores.item())
        scores.append(cos_scores.item())
        #break
    x = dict(zip(X, scores))
    #print(x)
    xr = dict(sorted(x.items(), key=operator.itemgetter(1), reverse=True))
    #print("***************************************")
    #print("***************************************")
    #print(xr)
    #print(xr.keys())
    return list(xr.keys())

In [20]:
from sentence_transformers import CrossEncoder, SentenceTransformer, util
if "cross-encoder" in args_model: model = CrossEncoder(args_model, max_length=256)
else: 
    model = SentenceTransformer(args_model)
    model.eval()

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/199 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/35.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [22]:
import time
from tqdm import tqdm
start = time.time()
for k in tqdm(run.keys()):
    if "cross-encoder" in args_model: run[k] = reranking_crossEncoder(k, run[k])
    else: run[k] = reranking_biEncoder(k, run[k])
    break
end = time.time()
print(end - start)

  0%|          | 0/6968 [00:01<?, ?it/s]

1.2715911865234375


In [23]:
print(output_run)

mmarco-Arabic-mMiniLML-bi-encoder-KD-v1run.arabic-msmarco-passage.bm25tuned.txt.tsv


In [ ]:
# Run reranker
with open(output_run, 'w', encoding='utf-8') as f_out:
    for k in run.keys():
        rank=1
        for j in run[k]:
            f_out.write(f'{k}\t{j}\t{rank}\n')
            rank =rank +1
print("Done!")

In [ ]:
!pip install -U datasets huggingface-hub -q
import huggingface_hub 
hf = huggingface_hub.HfFolder()
# to load the reranked list to your Hugging Face repository, enter your access token below. 
# and change the repository ID to yours.
#organization_dataset_id =""
access_token = "Hagging face access token"
hf.save_token(access_token)
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj=output_run,
    path_in_repo=output_run,
    repo_id=organization_dataset_id,
    repo_type="dataset",
)